In [1]:
import sub.plot as pl
import sub.functions as sb
from global_variables import gparam

gl = gparam()

In [2]:
condition = {
    # TF current
    'cur_tf':{'tf': 50.0e+3, 'turn': 16},
    
    # initial plasma profile
    'cur_ip':{'ip':-100.0e+3, 'r0':0.65, 'z0':0.0, 'radius':0.3},
    
    # PF currents
    'cur_pf':{'hcult16':0.0,'pf17t12':1.0e+3, 'pf26t18':1.5e+3,'pf4_1ab3_cc2':1.0e+3,'pf35_2':-0.5e+3, },
    #'cur_pf':{'hcult16':0.0,'pf17t12':1.0e+3, 'pf26t18':1.0e+3,'pf4_1ab3_cc2':0.0,'pf35_2':0.0, },
    
    # number of coefficients
    'num_dpr':2, # dp/df
    'num_di2':2, # di2/df
    
    # calculate flux (r, z): result is set to 'fl_val'.
    'fl_pos':{'flc8':(0.1985, 0.450), 'f_im':(0.1985, 0.0), 'flc17':(0.1985, -0.450),
              'fls1':(1.374, 0.450), 'fls5':(1.374, 0.0), 'fls9':(1.374, -0.481)},
    
    'resolution': gl.get_dmat_coarse(),
    }

In [3]:
# calculate plasma equilibrium
cond = sb.calc_equilibrium(condition, verbose=1)

0 loss:  907962.3711028895
1 loss:  29452.506780234573
2 loss:  9124.781505372932
3 loss:  4348.619808876687
4 loss:  1793.282425195591
5 loss:  1072.001323551718
6 loss:  546.1538636597121
7 loss:  388.2749155579161
8 loss:  296.0410394379462
9 loss:  222.78279847105065
10 loss:  173.27535680553817
11 loss:  163.47284718421585
12 loss:  102.49931635577651
13 loss:  105.24981584968779


In [ ]:
# calculated parameters
cond.keys()

In [ ]:
# total flux profile
pl.d_contour(cond['flux'])

In [ ]:
# Plasma current profile
pl.d_contour(cond['jt'])

In [ ]:
# pressure
pl.d_contour(cond['pressure'])

In [ ]:
# poloidal current
pl.d_contour(cond['pol_current'])

In [ ]:
pl.d_heatmap(cond['safty_factor'])

In [ ]:
# vessel 
pl.d_heatmap(cond['vessel'])

In [ ]:
# domain
pl.d_heatmap(cond['domain'])